In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.convblock1 = nn.Sequential(
            
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=(3, 3), bias=False),
            nn.ReLU(),
            
        )
      

        self.convblock2 = nn.Sequential(
            
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3, 3), bias=False),
            nn.ReLU(),
            
        )

        
        self.convblock3 = nn.Sequential(
            
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 3), bias=False),
            nn.ReLU(),
            
        )

        self.pool3 = nn.MaxPool2d(2, 2) 


        self.convblock4 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3, 3), bias=False),
            nn.ReLU(),

        )

        

        self.convblock5 = nn.Sequential(   
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3, 3), bias=False),
            nn.ReLU(),       
            
        )

        self.convblock6 = nn.Sequential(   
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3, 3), bias=False),
            nn.ReLU(),      
            
        )



        self.convblock7 = nn.Sequential(   
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3, 3),bias=False),
            nn.ReLU(),  

                      
        )

        self.convblock8 = nn.Sequential(   
            nn.Conv2d(in_channels=64, out_channels=10, kernel_size=(3, 3),bias=False),
                      
        )




    def forward(self, x):


        x = self.convblock1(x)   
        x = self.convblock2(x)
        x = self.convblock3(x)
        x = self.pool3(x)
        x = self.convblock4(x)
        x = self.convblock5(x)
        x = self.convblock6(x)
        x = self.convblock7(x)
        x = self.convblock8(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [9]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             144
              ReLU-2           [-1, 16, 26, 26]               0
            Conv2d-3           [-1, 32, 24, 24]           4,608
              ReLU-4           [-1, 32, 24, 24]               0
            Conv2d-5           [-1, 64, 22, 22]          18,432
              ReLU-6           [-1, 64, 22, 22]               0
         MaxPool2d-7           [-1, 64, 11, 11]               0
            Conv2d-8             [-1, 64, 9, 9]          36,864
              ReLU-9             [-1, 64, 9, 9]               0
           Conv2d-10             [-1, 64, 7, 7]          36,864
             ReLU-11             [-1, 64, 7, 7]               0
           Conv2d-12             [-1, 64, 5, 5]          36,864
             ReLU-13             [-1, 64, 5, 5]               0
           Conv2d-14             [-1, 6

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:84: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [10]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([                   
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!


In [11]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    correct = 0
    processed = 0
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()
        processed += len(data)
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
   

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [12]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 16):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    print('------------------------------------------')

  0%|          | 0/469 [00:00<?, ?it/s]

EPOCH: 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:84: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
loss=0.1418478637933731 batch_id=468 Accuracy=67.29: 100%|██████████| 469/469 [00:09<00:00, 47.15it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.1243, Accuracy: 9622/10000 (96.22%)

------------------------------------------
EPOCH: 2


loss=0.1062420979142189 batch_id=468 Accuracy=96.79: 100%|██████████| 469/469 [00:10<00:00, 46.75it/s]
  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0351, Accuracy: 9905/10000 (99.05%)

------------------------------------------
EPOCH: 14


loss=0.020847663283348083 batch_id=468 Accuracy=99.56: 100%|██████████| 469/469 [00:09<00:00, 48.03it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0264, Accuracy: 9928/10000 (99.28%)

------------------------------------------
EPOCH: 15


loss=0.010790611617267132 batch_id=468 Accuracy=99.58: 100%|██████████| 469/469 [00:09<00:00, 48.29it/s]



Test set: Average loss: 0.0323, Accuracy: 9906/10000 (99.06%)

------------------------------------------
